In [8]:
# Dependencies

import pandas as pd
#import numpy as np
#from sqlalchemy.ext.automap import automap_base
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Table,Column,Integer,String
import glob
import os
from sqlalchemy import MetaData
from sqlalchemy.orm import mapper
#from sqlalchemy import inspect


In [9]:
# SQL Alchemy setup

# Create engine that will allow us to communicate with database
engine=create_engine('sqlite:///scores_dbase.sqlite',echo=False)

In [10]:
#Creating session which is the middle ground to talk to our engine
Session=sessionmaker(bind=engine)
session=Session()

In [11]:
# Map which table in database will be related to each class
Base=declarative_base()

In [12]:
# Create a metadata instance
#A metadata is an object container that will store attributes and name of table
metadata=MetaData(engine)

#Define structure of table
class scores_table(object):
    def __init__(self, number, description, ref_des):
        self.state=state
        self.gender=gender
        self.avg_2009_mathScores=avg_2009_mathScores
        self.avg_2017_mathScores=avg_2017_mathScores
        self.mathScores_percentChange=mathScores_percentChange
        
    def __repr__(self):
        return f'{self.state, self.gender, self.avg_2009_mathScores, self.avg_2017_mathScores, self.mathScores_percentChange}'

    

In [13]:
#Declaring a table
#Defining a function that defines table, we define this function so that we can keep table names dynamic
#That is, we can have multiple tables in database and each table can have a different name

def table_definition(table_name):

#Define table structure, here table_name varies
#We want to make table_define available outside function so we declare it as a function attribute

    table_definition.table_define=Table(table_name,metadata,
    #Column('id',Integer,primary_key=True),
    Column('state',String),
    Column('gender',String),
    Column('avg_2009_mathScores',INT),
    Column('avg_2017_mathScores',INT),
    Column('mathScores_percentChange',INT)
    )

In [14]:
#Create table
#Note that we used the engine from function
metadata.create_all(engine)
    
#Use mapper to define components of class as well as table definition together at once
mapper(scores_table,table_definition.table_define,non_primary=True)


AttributeError: 'function' object has no attribute 'table_define'

In [19]:
#CREATING A DUMMY BLANK TABLE FOR PRIMARY MAPPER
#This avoids error: Class already has a primary mapper defined
#We made non_primary=True in table_definition function mapper
#This is the work around I could use, maybe there is another way

#Define table structure, here table_name varies
table_define_dummy=Table('dummy_table',metadata,
#Column('id',Integer,primary_key=True),
Column('state',String),
Column('gender',String),
Column('avg_2009_mathScores',int),
Column('avg_2017_mathScores',int),
Column('mathScores_percentChange',int)
)

#Create table
metadata.create_all(engine)

#Use mapper to define components of class as well as table definition together at once
mapper(scores_table,table_define_dummy)

AttributeError: type object 'int' has no attribute '_set_parent_with_dispatch'

In [20]:
#This function will create a separate table for each csv file, if you have multiple csv files
#Name of table will be extracted from file name. File name contains product name. 
#Each table will be identified by product name
# It will read each excel file in the folder and insert bom into table
def create_table(folder_of_files):
    
#Get list of files in folder
    files=glob.glob(os.path.join(folder_of_files,"*.csv"))
    
    
#Loop through all files in list
    for file_name in files:
        
        #Read file into dataframe
        csv_data=pd.read_csv(file_name)
        
        #Convert dataframe to list and store in same variable
        csv_data=csv_data.values.tolist()
        
        #Get table name from file name. This will be our table name. 
        table_name_from_file=file_name.split('/')[8][:-4]
        
        #Use table_definition function to define table structure
        table_definition(table_name_from_file)
        
        

In [21]:
#Loop through list of lists, each list in create_bom_table.xls_data is a row

for row in csv_data:
            
    #Each element in the list is an attribute for the table class
    #Iterating through rows and inserting into table
            ins=table_definition.table_define.insert().values(
            product_id=row[0],station=row[1],time=row[2],result=row[3],employee_id=row[4])
            conn=engine.connect()
            conn.execute(ins)

NameError: name 'csv_data' is not defined

In [22]:
#Calling function, argument is path of folder where all CSV files are stored
create_table("data/rawdata")